# from google.colab import drive
# drive.mount("/content/drive")

## Import Modules

In [15]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# import tensorflow as tf
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, GRU, Dense, Concatenate, Flatten
# from tensorflow.keras.optimizers import Adam
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics
import matplotlib.pyplot as plt


In [2]:
# tf.config.list_physical_devices('GPU')

In [16]:
#Load Dataset
# data_path = '/content/drive/MyDrive/merged_weather_marine_data.csv'
# data=pd.read_csv(data_path)

#data_path = './Weather-Dataset/merged_weather_marine_data.csv'
# ini yg bener
data_path = './Weather-Dataset/sail_decision_2021-2024.csv'
data = pd.read_csv(data_path)

In [4]:
# visualization
def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values
    summ['%missing'] = df.isnull().sum().values / len(df)* 100
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    return summ


In [5]:
# #Cleaning and Preprocessing
# print(data.describe())

# #column temperature (min, max, avg), precipitation, snow, wind(speed, direction, peak gust), pressure, sunshine duration
# print(data.columns)

#melihat info descriptive statistic
print(data.describe())

#melihat banyaknya missing value
print(data.isnull().sum())

# #melihat banyaknya entri unique
# print(data.nunique())


       temperature_2m (°C)  relative_humidity_2m (%)  dew_point_2m (°C)  \
count         26304.000000              26304.000000       26304.000000   
mean             27.655056                 77.906060          23.403665   
std               0.975380                  5.904932           0.954461   
min              23.500000                 54.000000          17.600000   
25%              27.000000                 74.000000          23.000000   
50%              27.700000                 78.000000          23.600000   
75%              28.400000                 82.000000          24.000000   
max              31.100000                 95.000000          26.200000   

       apparent_temperature (°C)  precipitation (mm)     rain (mm)  \
count               26304.000000        26304.000000  26304.000000   
mean                   31.119514            0.231858      0.231858   
std                     1.929716            0.720542      0.720542   
min                    24.600000            

In [17]:
# Menentukan kolom-kolom yang bertipe object
object_cols = data.select_dtypes(include=['object']).columns
df = data.drop(columns=object_cols)

df = df.dropna()   #58 row is dropped
#df['sail_decision'] = 1
print(df.columns)
print(df.shape)
binary_label = df['sailing_decision'].values
df = df.drop(columns=['sailing_decision'])

Index(['temperature_2m (°C)', 'relative_humidity_2m (%)', 'dew_point_2m (°C)',
       'apparent_temperature (°C)', 'precipitation (mm)', 'rain (mm)',
       'weather_code (wmo code)', 'pressure_msl (hPa)', 'cloud_cover (%)',
       'wind_speed_100m (km/h)', 'wind_direction_100m (°)',
       'wind_gusts_10m (km/h)', 'is_day ()', 'direct_radiation (W/m²)',
       'daily_weather_code (wmo code)', 'wave_height (m)',
       'wave_direction (°)', 'wave_period (s)', 'wind_wave_height (m)',
       'wind_wave_direction (°)', 'wind_wave_period (s)',
       'wind_wave_peak_period (s)', 'swell_wave_height (m)',
       'swell_wave_direction (°)', 'swell_wave_period (s)',
       'swell_wave_peak_period (s)', 'ocean_current_velocity (km/h)',
       'ocean_current_direction (°)', 'sailing_decision'],
      dtype='object')
(12521, 29)


In [7]:
summary(df).style.background_gradient(cmap='Blues')

data shape: (12521, 28)


,data type,#missing,%missing,#unique,min,max
temperature_2m (°C),float64,0,0.000000,69,23.500000,30.900000
relative_humidity_2m (%),int64,0,0.000000,40,54.000000,95.000000
dew_point_2m (°C),float64,0,0.000000,79,17.600000,26.000000
apparent_temperature (°C),float64,0,0.000000,117,24.600000,36.700000
precipitation (mm),float64,0,0.000000,75,0.000000,11.400000
rain (mm),float64,0,0.000000,75,0.000000,11.400000
weather_code (wmo code),int64,0,0.000000,10,0.000000,65.000000
pressure_msl (hPa),float64,0,0.000000,121,1003.900000,1016.100000
cloud_cover (%),int64,0,0.000000,101,0.000000,100.000000
wind_speed_100m (km/h),float64,0,0.000000,486,0.000000,53.500000


In [18]:
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(df)

steps = 168
predict_steps = 1
# Create sequences
inp = []
out = []
binary = []
for i in range(len(data_scaled) - (steps + predict_steps - 1)):
    inp.append(data_scaled[i:i+steps])
    out.append(data_scaled[i+steps:i+steps+predict_steps])
    binary.append(binary_label[i+steps:i+steps+predict_steps])

inp = np.asarray(inp)
out = np.asarray(out)
binary = np.asarray(binary)

# Train-test split
train_test_ratio = 0.7
train_size = int(train_test_ratio * inp.shape[0])

# Split data
x_train_tseries = inp[:train_size, :, :]
x_test_tseries = inp[train_size:, :, :]
y_train_tseries = out[:train_size, :, :]
y_test_tseries = out[train_size:, :, :]
y_train_binary = binary[:train_size, :]
y_test_binary = binary[train_size:, :]

print("x_train_tseries shape:", x_train_tseries.shape)
print("x_test_tseries shape:", x_test_tseries.shape)
print("y_train_binary shape:", y_train_binary.shape)
print("y_test_binary shape:", y_test_binary.shape)

x_train_tseries shape: (8647, 168, 28)
x_test_tseries shape: (3706, 168, 28)
y_train_binary shape: (8647, 1)
y_test_binary shape: (3706, 1)


**Non seq2seq without static data, output only binary**

**Non Seq2seq with static data**

In [9]:
# # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # print(f"Using device: {device}")

# device = torch.device("cpu")

# class EncoderGRU(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.2):
#         super(EncoderGRU, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.gru = nn.GRU(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)

#     def forward(self, input_seq, hidden=None):
#         output, hidden = self.gru(input_seq, hidden)
#         return output, hidden

# class DecoderGRU(nn.Module):
#     def __init__(self, output_size, hidden_size, num_layers=1, dropout=0.2):
#         super(DecoderGRU, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.gru = nn.GRU(output_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
#         self.fc = nn.Linear(hidden_size, output_size)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, input_seq, hidden):
#         output, hidden = self.gru(input_seq, hidden)
#         output = self.fc(output)
#         output_bin = self.sigmoid(output)
#         return output_bin, hidden

# class Seq2Seq(nn.Module):
#     def __init__(self, encoder, decoder):
#         super(Seq2Seq, self).__init__()
#         self.encoder = encoder
#         self.decoder = decoder

#     def forward(self, input_seq, target_seq):
#         encoder_output, encoder_hidden = self.encoder(input_seq)

#         # Initialize decoder hidden state with encoder final hidden state
#         decoder_hidden = encoder_hidden

#         # Initialize decoder input with the first element of target_seq
#         decoder_input = target_seq[:, 0, :].unsqueeze(1)

#         outputs = []
#         for t in range(1, target_seq.size(1)):
#             decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
#             outputs.append(decoder_output)
#             decoder_input = decoder_output

#         outputs = torch.cat(outputs, dim=1)
#         return outputs

In [10]:


# # # Example usage
# # # input_size = 5
# # # hidden_size = 10
# # # output_size = 3
# # # num_layers = 1
# # # dropout = 0.2

# x_train_tseries = torch.tensor(x_train_tseries, dtype=torch.float32).to(device)
# # y_train_tseries = torch.tensor(y_train_tseries, dtype=torch.float32).to(device)
# x_test_tseries = torch.tensor(x_test_tseries, dtype=torch.float32).to(device)
# # y_test_tseries = torch.tensor(y_test_tseries, dtype=torch.float32).to(device)
# y_train_binary = torch.tensor(y_train_binary, dtype=torch.float32).to(device)
# y_test_binary = torch.tensor(y_test_binary, dtype=torch.float32).to(device)
# print(y_train_binary.shape)

# input_size = x_train_tseries.shape[2] #num input feature
# hidden_size = 64
# output_size = 1 #y_train_binary.shape[1]
# num_layers = 1
# dropout = 0.2

# encoder = EncoderGRU(input_size, hidden_size, num_layers, dropout).to(device)
# decoder = DecoderGRU(output_size, hidden_size, num_layers, dropout).to(device)
# model = Seq2Seq(encoder, decoder).to(device)

# # # # Dummy input and target sequences, nanti ini diganti
# # # input_seq = torch.randn(32, 10, input_size)  # (batch_size, seq_length, input_size)
# # # target_seq = torch.randn(32, 10, output_size)  # (batch_size, seq_length, output_size)


# # # Initialize model, loss function, and optimizer
# criterion = nn.BCELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # # Training loop
# num_epochs = 500
# for epoch in range(num_epochs):
#     model.train()
#     optimizer.zero_grad()
    
#     # Expand y_train_binary to match the model's expected input shape
#     y_train_binary_expanded = y_train_binary.unsqueeze(2)
#     print(y_train_binary_expanded.shape) #(samples,168,1)
#     # Ensure the output sequence length matches the target sequence length
#     outputs = model(x_train_tseries, y_train_binary_expanded)
    
#     # outputs = outputs[:, :predict_steps, :]
#     print(outputs.shape) #(samples,167,1)
#     # Calculate loss
#     loss = criterion(outputs, y_train_binary)
#     loss.backward()
#     optimizer.step()

#     if epoch % 10 == 0:
#         print(f'Epoch {epoch}, Loss: {loss.item()}')

# # # model.eval()  #for regression
# # # with torch.no_grad():
# # #     test_outputs = model(x_test_tseries, y_test_tseries)
# # #     test_loss = criterion(test_outputs, y_test_tseries[:, 1:, :])
# # #     print(f'Test Loss: {test_loss.item()}')

# # #     mae_metric = torchmetrics.MeanAbsoluteError().to(device)
# # #     test_mae = mae_metric(test_outputs, y_test_tseries[:, 1:, :])
# # #     print(f'Test MAE: {test_mae.item()}')

# model.eval()
# with torch.no_grad():
#     test_outputs = model(x_test_tseries, y_test_binary.unsqueeze(2))
#     test_loss = criterion(test_outputs.squeeze(2), y_test_binary)
#     print(f'Test Loss: {test_loss.item()}')

#     # Calculate accuracy
#     test_pred = (test_outputs.squeeze(2) > 0.5).float()
#     accuracy = (test_pred == y_test_binary).float().mean()
#     print(f'Test Accuracy: {accuracy.item()}')


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

x_train_tseries = torch.tensor(x_train_tseries, dtype=torch.float32).to(device)
x_test_tseries = torch.tensor(x_test_tseries, dtype=torch.float32).to(device)
y_train_binary = torch.tensor(y_train_binary, dtype=torch.float32).to(device)
y_test_binary = torch.tensor(y_test_binary, dtype=torch.float32).to(device)


# Define the models and training process
class EncoderGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, dropout=0.2):
        super(EncoderGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, dropout=dropout, batch_first=True)

    def forward(self, input_seq, hidden=None):
        output, hidden = self.gru(input_seq, hidden)
        return output, hidden

class DecoderGRU(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers=1, dropout=0.2):
        super(DecoderGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(output_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_seq, hidden):
        output, hidden = self.gru(input_seq, hidden)
        output = self.fc(output)
        output = torch.sigmoid(output)  # Apply sigmoid for binary classification
        return output, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, target_seq):
        encoder_output, encoder_hidden = self.encoder(input_seq)

        decoder_hidden = encoder_hidden
        decoder_input = target_seq[:, 0, :].unsqueeze(1)

        outputs = []
        for t in range(target_seq.size(1)):  # Ensure it runs for the length of the target sequence
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs.append(decoder_output)
            decoder_input = decoder_output

        outputs = torch.cat(outputs, dim=1)
        return outputs

input_size = x_train_tseries.shape[2]
hidden_size = 10
output_size = 1  # Binary output (0 or 1)
num_layers = 1
dropout = 0.2

encoder = EncoderGRU(input_size, hidden_size, num_layers, dropout).to(device)
decoder = DecoderGRU(hidden_size, output_size, num_layers, dropout).to(device)
model = Seq2Seq(encoder, decoder).to(device)

# Initialize model, loss function, and optimizer
criterion = nn.BCELoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Expand y_train_binary to match the model's expected input shape
    y_train_binary_expanded = y_train_binary.unsqueeze(2).expand(-1, -1, output_size)
    
    # Ensure the output sequence length matches the target sequence length
    outputs = model(x_train_tseries, y_train_binary_expanded)
    outputs = outputs[:, :predict_steps, :]
    
    # Calculate loss
    loss = criterion(outputs.squeeze(2), y_train_binary)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Evaluation on test data
model.eval()
with torch.no_grad():
    y_test_binary_expanded = y_test_binary.unsqueeze(2).expand(-1, -1, output_size)
    test_outputs = model(x_test_tseries, y_test_binary_expanded)
    test_outputs = test_outputs[:, :predict_steps, :]
    test_loss = criterion(test_outputs.squeeze(2), y_test_binary)
    print(f'Test Loss: {test_loss.item()}')

    # Calculate accuracy
    test_pred = (test_outputs.squeeze(2) > 0.5).float()
    accuracy = (test_pred == y_test_binary).float().mean()
    print(f'Test Accuracy: {accuracy.item()}')

Using device: cuda
Epoch 0, Loss: 0.7089789509773254


/home/serverai/miniconda3/envs/uas-ai/lib/python3.9/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 10, Loss: 0.6833817362785339
Epoch 20, Loss: 0.6639968752861023
Epoch 30, Loss: 0.6481136083602905
Epoch 40, Loss: 0.633327841758728
Epoch 50, Loss: 0.6177583932876587
Epoch 60, Loss: 0.6001420617103577
Epoch 70, Loss: 0.5795164704322815
Epoch 80, Loss: 0.5555638074874878
Epoch 90, Loss: 0.5292985439300537
Epoch 100, Loss: 0.503461480140686
Epoch 110, Loss: 0.4814114272594452
Epoch 120, Loss: 0.4640806019306183
Epoch 130, Loss: 0.4495459794998169
Epoch 140, Loss: 0.43615132570266724
Epoch 150, Loss: 0.4231993556022644
Epoch 160, Loss: 0.4103987216949463
Epoch 170, Loss: 0.39757686853408813
Epoch 180, Loss: 0.3845936059951782
Epoch 190, Loss: 0.3713834583759308
Epoch 200, Loss: 0.35792186856269836
Epoch 210, Loss: 0.34420979022979736
Epoch 220, Loss: 0.3302748501300812
Epoch 230, Loss: 0.316169410943985
Epoch 240, Loss: 0.30196481943130493
Epoch 250, Loss: 0.28774088621139526
Epoch 260, Loss: 0.2735758423805237
Epoch 270, Loss: 0.25953811407089233
Epoch 280, Loss: 0.24568097293376

In [14]:
DATA.predict

'cuda'

Using device: cuda
torch.Size([100, 10, 5])
torch.Size([100, 3])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1000x1 and 100x100)